# COMP9033 - Data Analytics Lab 10: Association rule mining
## Introduction

In this lab, we're going to look at association rule mining for grocery data. At the end of the lab, you should be able to:

- Find frequently occurring itemsets using the FPGrowth algorithm.
- Compute the support of an association rule.
- Compute the confidence of an association rule.
- Sort association rules by support and confidence.

### Getting started

Let's start by importing the packages we'll need. As usual, we'll import `pandas` for exploratory analysis, but this week we're also going to use `pyspark`, a Python package that wraps Apache Spark and makes its functionality available in Python. Spark also supports frequent itemset generation using the FPGrowth algorithm, so we'll import this functionality too.

In [ ]:
import itertools
import pandas as pd
import pyspark

from pyspark.mllib.fpm import FPGrowth

First, let's initialise a [`SparkContext`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.SparkContext) object, which will represent our connection to the Spark cluster. To do this, we must first specify the URL of the master node to connect to. As we're only running this notebook for demonstration purposes, we can just run the cluster locally, as follows:

In [ ]:
sc = pyspark.SparkContext(master='local[*]')

> **Note:** By specifying `master='local[*]'`, we are instructing Spark to run with as many worker threads as there are logical cores available on the host machine. Alternatively, we could directly specify the number of threads, e.g. `master='local[4]'` to run four threads. However, we need to make sure to specify at least *two* threads, so that there is one available for resource management and at least one available for data processing.

Next, let's load the data. Write the path to your `groceries.csv` file in the cell below:

In [ ]:
path = 'data/groceries.csv'

We can load the data using the [`textFile`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.SparkContext.textFile) method of the `SparkContext` we have created, like this:

In [ ]:
transactions = sc.textFile(path)

However, `textFile` only has the effect of loading the raw data into a [Spark RDD](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD). Unlike `pandas`, it doesn't parse the CSV structure of the file and extract the individual fields. We can see this directly by examining the first few entries of the RDD using its [`take`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.take) method:

In [ ]:
transactions.take(5) # Take the first five entries

As can be seen, the data consists of a collection of transactions from a supermarket, where each row corresponds to a transaction and the items in a row correspond to the items that were purchased in that transaction.

## Preprocessing

The rows in the `transactions` RDD consist of the raw string data from the `groceries.csv` file, i.e. each row is an unparsed CSV string. Before we can mine frequent itemsets, we'll need to parse these strings into lists of items.

Parsing CSV strings is a relatively trivial operation in standard Python - we just need to call the [`split`](https://docs.python.org/2/library/stdtypes.html#str.split) method. For instance, we can split the string `'hello,world'` into the list of strings `['hello', 'world']` as follows:

In [ ]:
my_csv = 'hello,world'
my_csv.split(',')

To apply the same logic to the data in the `transactions` RDD, all we need to do is apply the `split` function to each line. We can do this using the [`map`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.map) method of the RDD, as follows:

In [ ]:
transactions = transactions.map(lambda line: line.split(','))
transactions.take(5)

## Association rule mining

Next, let's mine our transaction data to find interesting dependencies between itemsets. While we could do this in a brute force manner, as the number of transactions and items grows larger, the approach becomes significantly more computationally demanding. Instead, let's use frequent itemset generation to first generate a set of frequently occurring sets of items and then mine these for association rules. This approach is much more computationally efficient than a brute force strategy.

### Frequent itemset generation

While there are a number of approaches available for mining frequently occuring itemsets (e.g. Apriori, Eclat), Spark supports the [`FPGrowth`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.fpm.FPGrowth) algorithm directly. To run the algorithm on our set of transactions, we need to specify two parameters:

1. `minSupport`: A minimum support threshold, used to filter out itemsets that don't occur frequently enough.
2. `numPartitions`: The number of partitions used by parallel FPGrowth (affects performance only, not accuracy).

Let's set the minimum support level at 1% and use ten partitions. We can then train a frequent itemset model using the [`train`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.fpm.FPGrowth.train) method of the `FPGrowth` class, as follows:

In [ ]:
model = FPGrowth.train(transactions, minSupport=0.01, numPartitions=10)

We can extract the most frequent itemsets from the model using its [`freqItemsets`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.fpm.FPGrowthModel.freqItemsets) method, as follows:

In [ ]:
itemsets = model.freqItemsets().collect() # Call collect to reduce the RDD to a Python list

Let's examine the results of the modelling:

In [ ]:
print('Found %d frequent itemsets' % len(itemsets))
itemsets[:5]

As can be seen, the FPGrowth algorithm has identified 332 frequent itemsets in the transaction history. Each itemset is represented by a [`FreqItemset`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.fpm.FPGrowth.FreqItemset) object, which has two properties:

1. `items`: The set of items in the frequently occurring itemset.
2. `freq`: The number of times the set of items has occurred in the transaction history.

### Association rule mining

Association rule mining is supported by Spark in both its Java and Scala APIs. However, Python support is not currently available (although is due in the next release of Spark).

Instead, let's compute the association rules directly using Python. This involves a little more coding than in previous labs, but the general idea is the same as covered in class. To start, let's create a map of each frequent itemset to its count in the transaction history:

In [ ]:
counts_x = {tuple(sorted(itemset.items)): itemset.freq for itemset in itemsets} # items -> frequency

This dictionary simply maps the items in each frequent itemset to the number of times those items have appeared in the transaction history, i.e. it measures $\text{count}(X)$ for each frequent itemset $X$.

In [ ]:
counts_x

Next, let's generate every possible antecedent-consequent pair that can be made from the frequent itemsets and map them to their frequencies:

In [ ]:
counts_xy = {}
for itemset in itemsets:
    items = sorted(itemset.items)
    for i in range(len(items)):
        for x in itertools.combinations(items, i+1):
            y = tuple(sorted([i for i in items if i not in x]))
            if y:
                counts_xy.setdefault(x, {})
                counts_xy[x][y] = itemset.freq

This dictionary simply maps all possible subsets of the frequent itemsets to their frequencies in the transaction history, i.e. it measures $\text{count}(X \cup Y)$ for each possible antecedent $X$ and consequent $Y$.

In [ ]:
counts_xy

Finally, we can generate a set of association rules by compute the support and confidence for each antecedent-consequent pair, i.e.

\begin{align}
\def \tcount{\text{count}}
\def \support{\text{support}}
\def \confidence{\text{confidence}}
\support(X \Rightarrow Y) = \frac{\tcount(X \cup Y)}{n},\\
\\
\confidence(X \Rightarrow Y) = \frac{\tcount(X \cup Y)}{\tcount(X)}.
\end{align}

Using the `counts_x` and `counts_xy` dictionaries above, this is quite simple to do:

In [ ]:
n = transactions.count() # Compute the total number of transations

rules = []
for x in counts_xy:
    count_x = counts_x[x]

    for y in counts_xy[x]:
        count_xy = counts_xy[x][y]

        support = 1.0 * count_xy / n
        confidence = 1.0 * count_xy / count_x

        rules.append({'antecedent': x,
                      'consequent': y,
                      'support': support,
                      'confidence': confidence})

We can use `pandas` to more easily explore the results:

In [ ]:
df = pd.DataFrame(rules, columns=['antecedent', 'consequent', 'support', 'confidence'])
df.sort_values(['support', 'confidence'], ascending=False, inplace=True)
df.head(10)

As can be seen, many of rules with the highest level of support and confidence relate to whole milk. This is partly because of the distribution of different kinds of items in the transaction history, i.e. relatively few occur very commonly while most occur very irregularly:

In [ ]:
item_counts = transactions.flatMap(lambda item: item). \
                           map(lambda item: (item, 1)). \
                           reduceByKey(lambda x, y: x + y). \
                           collectAsMap()
item_counts = pd.Series(item_counts)

item_counts.hist(bins=20);

We could attempt to merge some of the more rarely occuring items (e.g. `'baby food'`) with other rarely occuring but related items (e.g. `'baby cosmetics'`) to force the algorithm to generate more meaningful rules (i.e. use feature transformation).

In [ ]:
item_counts.sort_values(ascending=True).head(10)